# Logit prediction using Maximum Likelihood Estimation 

Goal: predicting variable `high_ph` with `acid` and `base` variables.

Method: brute force maximum likelihood estimation.

In [436]:
import pandas as pd
import numpy as np
import random

Use the sigmoid canonical link function $ln(\frac{p}{(1-p)})$ which equals the logit $z = \alpha + \beta_1 + \beta_2$ to find the probability $p = \frac{1}{1+e^{-z}}$ of a target variable.

Maximize the log-likelihood function $\Sigma_{i} [Y_i * ln(p_i) + (1-Y_i) * ln(1-p_i)]$ to find the best parameters for the logit.

In [437]:
def logit(df, alpha, beta1, beta2):
    return alpha + beta1 * df.acid + beta2 * df.base

def probability(df):
    return 1/(1 + np.e ** - df.logit)

def log_likelihood(df):
    return ((df.high_ph * np.log(df.probability)) + ((1 - df.high_ph) * np.log(1 - df.probability)))  

In [438]:
data = {'acid': [0.34, 0.44, 0.37, 0.57, 0.12, 0.93, 0.17, 0.70, 0.39, 0.66, 0.05, 0.73],
        'base': [0.84, 0.21, 0.77, 0.14, 0.81, 0.44, 0.78, 0.12, 0.99, 0.23, 0.82, 0.24],
        'high_ph': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
        }

df = pd.DataFrame(data)
df

,acid,base,high_ph
0,0.34,0.84,0
1,0.44,0.21,1
2,0.37,0.77,0
3,0.57,0.14,1
4,0.12,0.81,0
5,0.93,0.44,1
6,0.17,0.78,0
7,0.70,0.12,1
8,0.39,0.99,0
9,0.66,0.23,1


Generate many sets of 3 random numbers which will be the candidates for the logit's $\alpha, \beta_1 and \beta_2$ parameters.

In [439]:
random_params = []
n_sets = 1000
for i in range(n_sets):
    random_params.append([random.randrange(-100, 100)/10, random.randrange(-100, 100)/10, random.randrange(-100, 100)/10])

For each parameter set, add up the resulting log-likelihood values and find the parameter set which generated the maximum value.

In [440]:
sum_of_log_likelihood = []
for param_set in random_params:
    df['logit'] = logit(df, *param_set) 
    df['probability'] = probability(df)
    df['log_likelihood'] = log_likelihood(df)
    sum_of_log_likelihood.append(df['log_likelihood'].sum())

best_params = random_params[np.argmax(sum_of_log_likelihood)]
best_params

[-0.4, 9.8, -9.5]

Use the selected parameters to predict the probability of `high_ph`.

In [441]:
z = best_params[0] + best_params[1] * df.acid + best_params[2] * df.base
high_ph_pred = np.e ** z / (1 + np.e ** z)
pd.DataFrame({'high_ph_pred': round(high_ph_pred, 2), 'high_ph': df.high_ph})

,high_ph_pred,high_ph
0,0.01,0
1,0.87,1
2,0.02,0
3,0.98,1
4,0.00,0
5,0.99,1
6,0.00,0
7,1.00,1
8,0.00,0
9,0.98,1
